# Author: Demetri Papamichalis
# ANA 630 - Microproject 4 - wk4

# Intro to Machine Learning: 

# 1. Import Libraries

In [195]:
import numpy as np
import pandas as pd
import math
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib
matplotlib.matplotlib_fname()
import sympy as sp
import sys
import datetime
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split # Function to split arrays or matrices into random train and test subsets.
from sklearn.tree import DecisionTreeClassifier # Class capable of performing multi-class classification on a dataset.
import seaborn as sb
sb.set_style('whitegrid')
from collections import Counter

# 1.1 Check my directory

In [196]:
# gives current path directory
from pathlib import Path
print(Path.cwd())

C:\Users\dpapa_fxbb8d0


# 1.2 Import data file set

In [197]:
# read text file into pandas DataFrame
data = pd.read_csv("C:\\Users\dpapa_fxbb8d0\earthquakes.csv");
print(df)

                            time  latitude  longitude   depth  mag magType  \
0       1930-12-08T08:01:02.000Z   23.2610   120.2770   15.00  6.3      mw   
1       1930-12-03T18:51:47.000Z   18.2330    96.2980   10.00  7.4      mw   
2       1930-12-02T07:01:30.000Z   25.8540    98.3560   35.00  6.2      mw   
3       1930-11-28T07:32:56.000Z   18.7790  -106.7670   15.00  6.3      mw   
4       1930-11-25T19:02:53.000Z   35.0500   139.1290   15.00  6.9      mw   
...                          ...       ...        ...     ...  ...     ...   
797041  2018-09-01T01:14:38.230Z  -30.4830  -177.9279   43.90  4.3      mb   
797042  2018-09-01T01:07:59.120Z  -10.7558   124.3621   10.00  4.0      mb   
797043  2018-09-01T01:00:13.810Z   -5.5167   147.1735  217.56  4.6      mb   
797044  2018-09-01T00:27:11.440Z   46.8819   155.6566   10.00  4.3      mb   
797045  2018-09-01T00:00:47.980Z  -55.7508   -28.3561   10.00  4.8      mb   

        nst    gap   dmin   rms  ...                   updated 

In [198]:
data.shape

(797046, 22)

In [199]:
# Check labeled columns:
data.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

The data comes from the United States Geological Survey (USGS) earthquake catalog.It includes all earthquakes from 1930 until 2018, worldwide containing basic information such as location, time and magnitude.The data contains information that we will use via machine learning to predict whether an earthquake event of 1.0 magnitude or greater will take place. Our binary categorization was that any event that has a magnitude 1.0 or greater will be classified as "1", and any event less than 0 magnitude will be classified as "0".

# 2. Prepare/Clean Data

In [200]:
#donwload file to load in excel:
data.to_csv (r"C:\\Users\dpapa_fxbb8d0\earthquake.csv", index = False)

In [201]:
# Attributes list:
data.__dir__()

['T',
 '_AXIS_LEN',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '_AXIS_TO_AXIS_NUMBER',
 '_HANDLED_TYPES',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex

In [202]:
# view our data
data;

In [203]:
# view first row of data
data.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1930-12-08T08:01:02.000Z,23.261,120.277,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930-12-03T18:51:47.000Z,18.233,96.298,10.0,7.4,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1930-12-02T07:01:30.000Z,25.854,98.356,35.0,6.2,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930-11-28T07:32:56.000Z,18.779,-106.767,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1930-11-25T19:02:53.000Z,35.050,139.129,15.0,6.9,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [204]:
data.info;

In [205]:
# Summary statistics:
data.describe()

,latitude,longitude,depth,mag,nst,gap,dmin,rms,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
count,797046.000000,797046.000000,797041.000000,797046.000000,421658.000000,470629.000000,202838.000000,614095.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,19.038314,-12.911692,53.726903,3.884296,33.170883,141.002356,1.757801,0.763249,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,29.219884,118.010192,99.510254,0.911611,55.796692,81.447357,3.734481,0.477974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,-84.422000,-179.999000,-4.900000,2.500000,0.000000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,-4.727000,-118.086000,10.000000,3.000000,8.000000,75.000000,0.082880,0.420000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,29.826000,-52.385000,23.000000,4.000000,16.000000,125.000000,0.583000,0.810000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,39.405000,120.892000,45.620000,4.600000,35.000000,195.900000,1.973000,1.070000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,87.221000,180.000000,735.800000,9.500000,934.000000,360.000000,127.420000,69.320000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [206]:
# check types of variables within our data:
data.dtypes

time                object
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place              float64
type               float64
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status             float64
locationSource     float64
magSource          float64
dtype: object

In [207]:
#...or another way
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 797046 entries, 0 to 797045
Data columns (total 22 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   time             797046 non-null  object 
 1   latitude         797046 non-null  float64
 2   longitude        797046 non-null  float64
 3   depth            797041 non-null  float64
 4   mag              797046 non-null  float64
 5   magType          796940 non-null  object 
 6   nst              421658 non-null  float64
 7   gap              470629 non-null  float64
 8   dmin             202838 non-null  float64
 9   rms              614095 non-null  float64
 10  net              797046 non-null  object 
 11  id               797046 non-null  object 
 12  updated          797046 non-null  object 
 13  place            0 non-null       float64
 14  type             0 non-null       float64
 15  horizontalError  0 non-null       float64
 16  depthError       0 non-null       floa

In [208]:
# show variables/columns:
data.columns

Index(['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'nst',
       'gap', 'dmin', 'rms', 'net', 'id', 'updated', 'place', 'type',
       'horizontalError', 'depthError', 'magError', 'magNst', 'status',
       'locationSource', 'magSource'],
      dtype='object')

In [209]:
data = data.drop(['time','magType','net','id','updated','place','type','horizontalError','depthError','magError','magNst','status','locationSource','magSource'], axis=1)
data.head()

,latitude,longitude,depth,mag,nst,gap,dmin,rms
0,23.261,120.277,15.0,6.3,NaN,NaN,NaN,NaN
1,18.233,96.298,10.0,7.4,NaN,NaN,NaN,NaN
2,25.854,98.356,35.0,6.2,NaN,NaN,NaN,NaN
3,18.779,-106.767,15.0,6.3,NaN,NaN,NaN,NaN
4,35.050,139.129,15.0,6.9,NaN,NaN,NaN,NaN


In [210]:
# Search for "NaN"/missing elements in dataset:
data.isnull().values.any()

True

In [211]:
# show how many missing entries in each column
data.isnull().sum()

latitude          0
longitude         0
depth             5
mag               0
nst          375388
gap          326417
dmin         594208
rms          182951
dtype: int64

In [213]:
# Search for NaN elements
data[data.isnull().any(axis=1)]

,latitude,longitude,depth,mag,nst,gap,dmin,rms
0,23.2610,120.2770,15.00,6.3,NaN,NaN,NaN,NaN
1,18.2330,96.2980,10.00,7.4,NaN,NaN,NaN,NaN
2,25.8540,98.3560,35.00,6.2,NaN,NaN,NaN,NaN
3,18.7790,-106.7670,15.00,6.3,NaN,NaN,NaN,NaN
4,35.0500,139.1290,15.00,6.9,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
797041,-30.4830,-177.9279,43.90,4.3,NaN,165.0,1.233,0.87
797042,-10.7558,124.3621,10.00,4.0,NaN,112.0,0.998,1.23
797043,-5.5167,147.1735,217.56,4.6,NaN,119.0,3.455,0.71
797044,46.8819,155.6566,10.00,4.3,NaN,94.0,6.370,1.21


In [214]:
data.shape

(797046, 8)

In [215]:
before_rows = data.shape[0] # Get rows dimension before dropping
print(before_rows)

797046


In [216]:
# Drop all rows with 'NaN' entries:
data = data.dropna()

In [217]:
after_rows = data.shape[0] # Get rows dimension after dropping
print(after_rows)

112192


In [218]:
# How many rows dropped?
before_rows - after_rows

684854

In [17]:
# Now, let's check if we have negative values in variables for out time and energy values
#(df_sorted<0).values.any()

In [219]:
# Check for duplicates:
data.duplicated()

104       False
107       False
108       False
111       False
113       False
          ...  
797014    False
797015    False
797020    False
797029    False
797040    False
Length: 112192, dtype: bool

In [220]:
# Summary statistics:
data.describe()

,latitude,longitude,depth,mag,nst,gap,dmin,rms
count,112192.000000,112192.000000,112192.000000,112192.000000,112192.000000,112192.000000,112192.000000,112192.000000
mean,32.486468,-118.717767,11.029965,2.953977,24.731683,155.238175,0.315327,0.237841
std,8.599509,22.847113,18.706551,0.431826,24.451835,102.116081,0.819742,0.601497
min,16.193000,-179.695500,-3.600000,2.500000,0.000000,0.000000,0.000000,0.000000
25%,19.573975,-122.797500,2.150750,2.640000,8.000000,60.000000,0.028240,0.090000
50%,35.100000,-118.925917,6.000000,2.830000,17.000000,128.000000,0.120000,0.180000
75%,37.797000,-115.797833,10.000000,3.140000,34.000000,249.000000,0.478700,0.290000
max,71.296200,179.575300,300.000000,7.200000,369.000000,360.000000,127.420000,64.290000


In [221]:
data.shape

(112192, 8)

# 3. Convert to a Classification Task

In [222]:
clean_data = data.copy() # Create a new DataFrame containing only the clean data
clean_data

,latitude,longitude,depth,mag,nst,gap,dmin,rms
104,37.477000,-118.114333,6.00,3.50,4.0,341.00,0.43200,2.8600
107,33.257000,-116.030000,6.00,3.37,2.0,354.00,1.34200,0.8000
108,33.451167,-116.163167,6.00,2.83,3.0,355.00,1.14600,0.5700
111,33.767500,-116.419167,6.00,3.20,3.0,357.00,0.82690,0.6100
113,34.549167,-116.201500,6.00,2.89,2.0,358.00,1.57000,0.8400
...,...,...,...,...,...,...,...,...
797014,40.385833,-124.601167,24.53,2.51,28.0,248.00,0.22030,0.1700
797015,19.407333,-155.433000,0.80,2.56,33.0,55.00,0.08607,0.1300
797020,18.819600,-67.246100,10.00,2.64,18.0,219.00,0.37710,0.3500
797029,39.205400,-114.957100,5.00,2.70,11.0,92.17,0.19300,0.2333


In [223]:
# Add a new column "strength" (magnitude of earthquake)to hold binary data based on
#the condition mag > 2.5 (2.5 is min value in dataset), multiply the boolean by 1 to yield
# integer values (1 or 0) - this becomes the target variable.
mag_min = clean_data['mag'].min()
clean_data['strength'] = (clean_data['mag'] > mag_min )*1
clean_data['strength']

104       1
107       1
108       1
111       1
113       1
         ..
797014    1
797015    1
797020    1
797029    1
797040    0
Name: strength, Length: 112192, dtype: int32

In [224]:
# store dependent variable "strength" to a new dataframe called 'y'
y = clean_data[['strength']]
y

,strength
104,1
107,1
108,1
111,1
113,1
...,...
797014,1
797015,1
797020,1
797029,1


# Sensor fusion

Variables time, latitude,longitude,depth,mag,nst,gap,dmin,rms are used to predict magnitude of earthquakes

In [247]:
# Grab the column labels to be used as features (predictor variables) in the
# machine learning model
    # first take out depedent variable "mag'
signals1 = clean_data[clean_data.columns[~clean_data.columns.isin(['mag'])]]
signals1

,latitude,longitude,depth,nst,gap,dmin,rms,strength
104,37.477000,-118.114333,6.00,4.0,341.00,0.43200,2.8600,1
107,33.257000,-116.030000,6.00,2.0,354.00,1.34200,0.8000,1
108,33.451167,-116.163167,6.00,3.0,355.00,1.14600,0.5700,1
111,33.767500,-116.419167,6.00,3.0,357.00,0.82690,0.6100,1
113,34.549167,-116.201500,6.00,2.0,358.00,1.57000,0.8400,1
...,...,...,...,...,...,...,...,...
797014,40.385833,-124.601167,24.53,28.0,248.00,0.22030,0.1700,1
797015,19.407333,-155.433000,0.80,33.0,55.00,0.08607,0.1300,1
797020,18.819600,-67.246100,10.00,18.0,219.00,0.37710,0.3500,1
797029,39.205400,-114.957100,5.00,11.0,92.17,0.19300,0.2333,1


In [248]:
#Now grab columns LABELS only (without grabbing "mag")
indepedent = indepedent.columns[0:7]
indepedent

Index(['latitude', 'longitude', 'depth', 'nst', 'gap', 'dmin', 'rms'], dtype='object')

In [249]:
# Create new DataFrame to hold only independent variables:
#'latitude', 'longitude', 'depth', 'nst', 'gap', 'dmin', 'rms',
# using the same name index.
X = clean_data[indepedent] # Named 'X' to signifiy independent variables
X

,latitude,longitude,depth,nst,gap,dmin,rms
104,37.477000,-118.114333,6.00,4.0,341.00,0.43200,2.8600
107,33.257000,-116.030000,6.00,2.0,354.00,1.34200,0.8000
108,33.451167,-116.163167,6.00,3.0,355.00,1.14600,0.5700
111,33.767500,-116.419167,6.00,3.0,357.00,0.82690,0.6100
113,34.549167,-116.201500,6.00,2.0,358.00,1.57000,0.8400
...,...,...,...,...,...,...,...
797014,40.385833,-124.601167,24.53,28.0,248.00,0.22030,0.1700
797015,19.407333,-155.433000,0.80,33.0,55.00,0.08607,0.1300
797020,18.819600,-67.246100,10.00,18.0,219.00,0.37710,0.3500
797029,39.205400,-114.957100,5.00,11.0,92.17,0.19300,0.2333


In [252]:
y.columns

Index(['strength'], dtype='object')

In [251]:
# View columns labels
print(X.columns)
print(len(X.columns)) # Number of features

Index(['latitude', 'longitude', 'depth', 'nst', 'gap', 'dmin', 'rms'], dtype='object')
7


# Perform test and train

Training Phase vs Testing Phase
In the training phase, the learning algorithm uses the training data to adjust the model's parameters to minimize errors. The output of this phase is a trained model.

In the testing phase, the trained model is applied to (previously "unseen") test data. The model's performance is then evaluated. The classifier model should perform well, approximately equal, on both the training data and the test data.

In [253]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#Output arguments: Two sets of independent variables (X) and dependent variables (y), one used to train the model and
# the second used to test the model -- all DataFrames.
# Function's input arguments: Independent variables X, and dependent variable y - both are Pandas DataFrames.
# test_size is set to 33% of the input data.
# random_state - controls the shuffling applied to the data before applying the split. 42 is a popular int random seed.

In [254]:
print(type(X_train))
print(type(X_test))
print(type(y_train))
print(type(y_test))
print(X_train.head())
y_train.describe() # Generates descriptive statistics.

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
         latitude   longitude   depth   nst    gap      dmin   rms
721603  18.228500  -67.208600  30.000  15.0  144.0  0.128459  0.09
378161  38.087667 -121.877167  20.235  56.0   60.0  0.089190  0.09
5830    33.782000 -116.318167   6.000   4.0  299.0  0.623900  0.56
603643  34.693500 -120.284333   2.341  31.0   61.0  0.042060  0.33
71074   46.210500 -122.190167   3.219  15.0   64.0  0.010690  0.16


,strength
count,75168.000000
mean,0.959717
std,0.196624
min,0.000000
25%,1.000000
50%,1.000000
75%,1.000000
max,1.000000


In [233]:
clean_data.dtypes

latitude     float64
longitude    float64
depth        float64
mag          float64
nst          float64
gap          float64
dmin         float64
rms          float64
strength       int32
dtype: object

# Train the classifier

In [256]:
strength_classifier = DecisionTreeClassifier(max_leaf_nodes=10, random_state=0) 
# Output is a decision tree classifier object.
# max_leaf_nodes is the stopping criteria for the tree induction, 
# default is 'unlimited' which can potentially over-fit
# the tree to the training data
# random_state argument is used for splitting the nodes - it's the "random seed", 
# i.e., controols the randomness of the estimator.

strength_classifier.fit(X_train, y_train)
# Train with the fit method of the object, i.e., the classifier will tune itself.

DecisionTreeClassifier(max_leaf_nodes=10, random_state=0)

In [260]:
type(strength_classifier)

sklearn.tree._classes.DecisionTreeClassifier

In [265]:
predictions[:20] # Display the first 10 values in those predictions

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [268]:
# Quick visual comparison to true values
y_test['strength'][:20]

784671    1
433166    1
569789    1
436417    1
766806    1
700078    1
23170     1
96962     1
70662     1
226431    1
766557    1
779455    1
17686     1
94611     1
772927    1
1696      1
700428    1
549314    1
645335    1
808       1
Name: strength, dtype: int32

# Predict on test data

In [259]:
predictions = strength_classifier.predict(X_test)

# Measure accuracy of the classifier

In [269]:
accuracy_score(y_true = y_test, y_pred = predictions)

0.9615654710458081